### Goal: adding in additional features such as resubmitted # and postal codes

In [1]:
import datasource as ds
import passencoder as pss
import pandasql as ps
import pandas as pd
u, p = pss.load_userpwd(r'C:\Users\razamo3\Documents\pass')



import numpy as np
import matplotlib.pyplot as plt

### Target DF - add in these features

In [2]:
finaldf = pd.DataFrame(columns=['cust_num','fafs_datekey','mtg_ind','resub_num', 'postal_code'])


### Initial Pull - SalesFunnel

```python
with ds.connect('dm',u,p) as con:
    q1 = """

      
       SELECT right(b.cust_num,9) as custnum, b.Resubmit_Num, b.Interest_Rate, a.*, b.Postal_Code_Key, b.Tds, b.Gds
              , b.Agreed_Term_Mm, b.Agreed_Amortizn_Mm, b.New2Canada_Ind, b.Non_Perm_Residt_Ind, b.Construction_Type_Name
               , b.Rental_Ind
       FROM [PBdm].[resl].[vw_FactSalesFunnel_Daily] as a 
       
       LEFT JOIN 
       
		  (SELECT * FROM [Pbdm].[resl].[vw_Applications_Daily] 
            WHERE Fullapp_First_Submitted_Date_Key > 20170430 AND Fullapp_First_Submitted_Date_Key < 20171001) as b
        
        ON a.Credit_Ref_Num = b.Credit_Ref_Num
          
        WHERE a.[Fullapp_First_Submitted_Date_Key] > 20170430 and a.[Fullapp_First_Submitted_Date_Key] < 20171001 
              AND a.Channel_Key = 50 and a.Product_Key in (67,68,69,70,71,72,73,74,100)

        """
    initialPull_table = con.read_sql(q1)

initialPull_table.to_pickle("tables/3a-initialpulltable.pkl")

```

In [6]:
initialPull_table = pd.read_pickle("tables/3a-initialpulltable.pkl")

### Clean up table to include cool features

In [8]:
features = ['custnum', 'Resubmit_Num', 'Interest_Rate', 'Total_Credit_Limit', 'Agreed_Term_Mm', 'Postal_Code_Key','Builder_Ind'
            , 'Tds', 'Ltv', 'Gds', 'Beacon_Score', 'Construction_Type_Name', 'Non_Perm_Residt_Ind', 'New2Canada_Ind'
            , 'Agreed_Amortizn_Mm', 'Rental_Ind']
df_join = initialPull_table[features]

In [9]:
df_join

,custnum,Resubmit_Num,Interest_Rate,Total_Credit_Limit,Agreed_Term_Mm,Postal_Code_Key,Builder_Ind,Tds,Ltv,Gds,Beacon_Score,Construction_Type_Name,Non_Perm_Residt_Ind,New2Canada_Ind,Agreed_Amortizn_Mm,Rental_Ind
0,640909539,0,3.09,269230.00,60,999999,0,48.96,0.95000,19.10,766,Detached,None,0,300,0
1,361698039,3,2.84,60000.00,24,999999,0,40.34,0.13953,39.10,802,Detached,None,0,300,0
2,638604126,5,2.45,540000.00,60,L0R1P0,0,43.73,0.65823,35.26,748,Detached,None,0,300,0
3,585406332,0,3.09,413465.00,48,L6Y0E3,0,33.79,0.80000,33.79,771,Detached,None,0,360,0
4,202474143,10,2.84,142500.00,60,999999,0,35.94,0.75000,17.13,637,Detached,None,0,300,0
5,664018326,3,2.45,517492.50,60,L6H0M8,0,112.36,0.75000,27.86,681,Detached,None,0,300,0
6,687151845,6,2.31,364000.00,36,999999,0,40.09,0.80000,22.72,725,Detached,None,0,300,0
7,592136631,2,2.64,390000.00,60,999999,0,57.97,0.60000,0.00,730,Duplex,None,0,360,1
8,683889642,0,2.54,288000.00,48,999999,0,42.73,0.80000,22.06,532,Detached,None,0,300,0
9,205593957,5,2.70,419900.00,60,L6Y3V1,0,39.97,0.95000,34.19,662,Detached,None,0,300,0


In [10]:
#Seperate custnums into their respective lists

apprvd_not_funded = initialPull_table[(initialPull_table.Approved_Apps == 1) & (initialPull_table.Funded_Count_Msr == 0)]
cv_apprvd_not_funded = apprvd_not_funded[['custnum', 'Total_Credit_Limit', 'Resubmit_Num', 'Interest_Rate']]

apprvd_and_fundedTD = initialPull_table[(initialPull_table.Approved_Apps == 1) & (initialPull_table.Funded_Count_Msr == 1)]
cv_apprvd_and_fundedTD = apprvd_and_fundedTD[['custnum', 'Total_Credit_Limit', 'Resubmit_Num', 'Interest_Rate']]

not_apprvd_not_funded = initialPull_table[(initialPull_table.Approved_Apps == 0) & (initialPull_table.Funded_Count_Msr == 0)]
cv_not_apprvd_not_funded = not_apprvd_not_funded[['custnum', 'Total_Credit_Limit', 'Resubmit_Num', 'Interest_Rate']]


In [1]:
not_apprvd_not_funded

NameError: name 'not_apprvd_not_funded' is not defined


### Need to find out where apprvd not funded people go...pull relavent CB data

In [11]:
#Convert lists to strings to pass custnums into appropriate CB table

apprvd_not_funded_lst = str(tuple(cv_apprvd_not_funded['custnum'].tolist()))
apprvd_and_fundedTD_lst = str(tuple(cv_apprvd_and_fundedTD['custnum'].tolist()))
combined_lst = (cv_apprvd_not_funded['custnum'].tolist()) + (cv_apprvd_and_fundedTD['custnum'].tolist())
combined_lst = str(tuple((combined_lst)))

```python
with ds.connect('dm',u,p) as con:
    q2 = """
        SELECT Date_Key, cust_num as custnum, count(Prod_Ind) AS NoOfMTGs, sum(limit) as TotalLimit
        FROM [PBdm].[dbo].[Credit_Bureau]
        where cust_num IN %s AND prod_ind = 'MTG'
        group by cust_num, Date_Key, Prod_Ind
        
        """
    df2a = con.read_sql(q2 % apprvd_not_funded_lst)

df2a.to_pickle("./3a_df2.pkl")
```

In [25]:
df2a = pd.read_pickle("tables/3a_df2.pkl")
df2b = pd.pivot_table(df2a, values = ['NoOfMTGs', 'TotalLimit'], index =['custnum'], columns =['Date_Key'])
df2 = df2b.fillna(0)
df2

NoOfMTGs                   TotalLimit                      
Date_Key  20160630 20170430 20171231   20160630   20170430   20171231
custnum                                                              
770094         3.0      2.0      2.0   658000.0   398000.0   398000.0
1431279        1.0      1.0      1.0   195000.0   195000.0   195000.0
2320605        4.0      3.0      2.0   532000.0   425000.0   324000.0
2629476        0.0      1.0      1.0        0.0    96000.0    96000.0
3466359        2.0      2.0      2.0   410000.0   410000.0   410000.0
3872925        1.0      0.0      1.0    75000.0        0.0   353000.0
4836474        1.0      1.0      1.0   356000.0   356000.0   356000.0
4974192        2.0      2.0      1.0   426000.0   426000.0   210000.0
5990724        2.0      2.0      2.0   228000.0   228000.0   228000.0
8334864        2.0      2.0      2.0   468000.0   468000.0   590000.0
8771463        2.0      2.0      2.0   107000.0   120000.0   120000.0
8945091        2.0      2.0      1.0   307000.0   307000.0   130000.0
9468378        1.0      2.0      2.0   330000.0   684000.0   684000.0
11566512       2.0      2.0      2.0   360000.0   360000.0   360000.0
11736315       1.0      1.0      1.0   220000.0   220000.0   220000.0
12146085       1.0      1.0      1.0    61000.0    61000.0    61000.0
12436659       3.0      3.0      4.0   716000.0   716000.0  1101000.0
14382117       2.0      2.0      3.0   538000.0   538000.0   888000.0
14970816       0.0      0.0      1.0        0.0        0.0   340000.0
16704387       1.0      1.0      1.0   200000.0   200000.0   200000.0
16911333       1.0      1.0      1.0   252000.0   252000.0   252000.0
19900467       1.0      1.0      1.0   250000.0   250000.0   250000.0
20116476       2.0      2.0      4.0  1002000.0  1002000.0  1086000.0
20253942       0.0      0.0      1.0        0.0        0.0    35000.0
20900196       1.0      1.0      2.0   306000.0   306000.0   608000.0
21671487       1.0      0.0      0.0   100000.0        0.0        0.0
22823622       1.0      1.0      1.0   250000.0   250000.0   250000.0
23917329       2.0      2.0      2.0   695000.0   695000.0   695000.0
23951601       2.0      1.0      1.0   368000.0   190000.0   190000.0
24714972       1.0      1.0      2.0   270000.0   270000.0   734000.0
...            ...      ...      ...        ...        ...        ...
711375894      0.0      0.0      2.0        0.0        0.0   751000.0
711397989      0.0      0.0      3.0        0.0        0.0   288000.0
711589617      0.0      0.0      2.0        0.0        0.0   645000.0
711637947      0.0      0.0      1.0        0.0        0.0   268000.0
711729153      0.0      0.0      1.0        0.0        0.0   225000.0
711999963      0.0      0.0      1.0        0.0        0.0   175000.0
712152153      0.0      0.0      1.0        0.0        0.0   256000.0
712343178      0.0      0.0      1.0        0.0        0.0   495000.0
712482363      0.0      0.0      2.0        0.0        0.0   353000.0
712519119      0.0      0.0      1.0        0.0        0.0    32000.0
712528101      0.0      0.0      2.0        0.0        0.0    92000.0
712620738      0.0      0.0      1.0        0.0        0.0   246000.0
712998855      0.0      0.0      2.0        0.0        0.0   171000.0
713127429      0.0      0.0      4.0        0.0        0.0   909000.0
713130084      0.0      0.0      1.0        0.0        0.0   195000.0
713187909      0.0      0.0      3.0        0.0        0.0   764000.0
713277468      0.0      0.0      2.0        0.0        0.0   359000.0
713651517      0.0      0.0      2.0        0.0        0.0   474000.0
713743452      0.0      0.0      1.0        0.0        0.0    80000.0
713807532      0.0      0.0      1.0        0.0        0.0   250000.0
714000384      0.0      0.0      2.0        0.0        0.0   158000.0
714016413      0.0      0.0      2.0        0.0        0.0   447000.0
714144078      0.0      0.0      2.0        0.0        0.0   454000.0
714464271      0.0  

### Start populating MTG_IND coloumn

In [14]:
#From inititialpull_table extract custnum, and indicators
df = initialPull_table[['custnum', 'Approved_Apps', 'Funded_Count_Msr']]

#create a df that has indicators of -1 (funded with comp), 0 (did not do anything), 1 (funded w/ TD)
df1 = pd.DataFrame(columns=['custnum','mtg_ind'])

#Start filling in mtg_ind coloumn with defined values
for i in range(0,df.shape[0],1):
    if (df.iloc[i,1] == 1 and df.iloc[i,2] == 1):
        df1 = df1.append({'custnum': df.iloc[i,0], 'mtg_ind': 1}, ignore_index=True)
        

In [18]:
#now append the people who fund with compettion (-1) and those who dont do anything (0)
    for i in range(0,df2.shape[0],1):
        if ((df2.iloc[i,2] > df2.iloc[i,1]) or ((df2.iloc[i,2] == df2.iloc[i,1] and df2.iloc[i,5] != df2.iloc[i,4]))):
            df1 = df1.append({'custnum': df.iloc[i,0], 'mtg_ind': -1}, ignore_index=True)
        elif ((df2.iloc[i,1] == df2.iloc[i,2] and df2.iloc[i,4] == df2.iloc[i,5])):
            df1 = df1.append({'custnum': df.iloc[i,0], 'mtg_ind': 0}, ignore_index=True)

In [19]:
df1[df1.mtg_ind==0]

,custnum,mtg_ind
22060,640909539,0
22061,361698039,0
22062,585406332,0
22063,202474143,0
22065,687151845,0
22066,683889642,0
22068,067805955,0
22069,378218637,0
22070,567295524,0
22071,611027892,0


### Get more customer info; age, byear, custsince, sex...etc

```python
with ds.connect('dm',u,p) as con:
    q2 = """
        SELECT CIFNum as custnum, BYear, Sex, CustSince 
        FROM [PBdm].[cif].[Customer]
        where CIFNum IN %s

        """
    customer_df = con.read_sql(q2 % combined_lst)


customer_df.to_pickle("tables/3b_customer_df.pkl")
```

In [30]:
customer_df = pd.read_pickle("tables/3b_customer_df.pkl")

#Clean up customer table
customer_df = customer_df.drop_duplicates()
customer_df

,custnum,BYear,Sex,CustSince
0,389008467,197305,M,199704
1,427023882,198810,M,200202
2,388498392,199207,F,199703
3,636350454,198206,F,199911
4,560147868,198104,M,200809
5,531948825,198708,M,200005
6,623882466,197705,M,198807
7,164730519,198407,F,199508
8,355694661,196411,M,199912
9,544040451,196412,F,199602


### LEFT JOIN TO CREATE FINAL_DF

In [21]:
#Convert the joining axis to integer values
df1 = df1.astype({'custnum': 'int'})
df_join = df_join.astype({'custnum':'int'})
customer_df = customer_df.astype({'custnum':'int'})

#Joing mtg_ind with reamining feautres
final_df =pd.merge(df1, df_join, how='left', on='custnum')

#Joining customer info to remaining table
final_df = pd.merge(final_df, customer_df, how ='left', on ='custnum')

### Clean up final table

In [22]:
#convert mtg_ind to int values
final_df = final_df.astype({'mtg_ind':'int'})

In [26]:
#Save final table
#final_df.to_pickle("tables/3c_final_df.pkl")

final_df = pd.read_pickle("tables/3c_final_df.pkl")

final_df

,custnum,mtg_ind,Resubmit_Num,Interest_Rate,Total_Credit_Limit,Agreed_Term_Mm,Postal_Code_Key,Builder_Ind,Tds,Ltv,Gds,Beacon_Score,Construction_Type_Name,Non_Perm_Residt_Ind,New2Canada_Ind,Agreed_Amortizn_Mm,Rental_Ind,BYear,Sex,CustSince
0,361698039,1,3,2.84,60000.00,24,999999,0,40.34,0.13953,39.10,802,Detached,None,0,300,0,193808.0,M,199806.0
1,202474143,1,10,2.84,142500.00,60,999999,0,35.94,0.75000,17.13,637,Detached,None,0,300,0,196111.0,M,199606.0
2,687151845,1,6,2.31,364000.00,36,999999,0,40.09,0.80000,22.72,725,Detached,None,0,300,0,197611.0,F,201510.0
3,205593957,1,5,2.70,419900.00,60,L6Y3V1,0,39.97,0.95000,34.19,662,Detached,None,0,300,0,197301.0,M,200010.0
4,67805955,1,1,2.84,87000.00,36,999999,0,23.53,0.42439,9.38,817,Detached,None,0,115,0,197101.0,M,198906.0
5,501998022,1,5,3.89,555695.23,48,T8N5K5,0,39.81,0.94274,19.05,649,Detached,None,0,281,0,197606.0,F,200510.0
6,611027892,1,0,3.14,190000.00,6,999999,0,41.16,0.76000,17.32,732,Detached,None,0,300,0,197005.0,M,200010.0
7,286878762,1,3,2.89,1125000.00,60,V5P1S1,0,42.82,0.75000,38.18,805,Detached,None,0,300,0,197911.0,M,199807.0
8,393390378,1,1,3.04,408000.00,48,999999,0,32.04,0.74182,27.68,708,Detached,None,0,360,0,198006.0,M,199807.0
9,364847571,1,3,2.59,799200.00,60,N0B1T0,0,42.94,0.80000,11.19,788,Detached,None,0,360,0,197312.0,M,199902.0


In [27]:
final_df.describe()

,custnum,mtg_ind,Resubmit_Num,Interest_Rate,Total_Credit_Limit,Agreed_Term_Mm,Builder_Ind,Tds,Ltv,Gds,Beacon_Score,New2Canada_Ind,Agreed_Amortizn_Mm,Rental_Ind,BYear,CustSince
count,2.738400e+04,27384.000000,27384.000000,27384.000000,2.738400e+04,27384.000000,27384.000000,27384.000000,27384.000000,27384.000000,27384.000000,27384.000000,27384.000000,27384.000000,26494.000000,26494.000000
mean,4.545612e+08,0.818288,2.344763,2.749456,2.891633e+05,45.351081,0.055726,41.292373,0.673425,28.309203,735.636905,0.004930,302.236744,0.074898,195778.926814,200126.910584
std,1.865882e+08,0.509552,2.045212,0.445289,1.832815e+05,18.699707,0.229396,68.841259,0.237645,68.921709,104.617374,0.070041,51.078324,0.263231,17587.826114,906.694523
min,8.934300e+04,-1.000000,0.000000,-2.100000,3.973690e+03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,190101.000000
25%,3.562048e+08,1.000000,1.000000,2.490000,1.738880e+05,24.000000,0.000000,30.660000,0.565700,16.020000,705.000000,0.000000,300.000000,0.000000,196409.000000,199510.000000
50%,4.879692e+08,1.000000,2.000000,2.690000,2.550000e+05,60.000000,0.000000,37.215000,0.750000,23.940000,756.000000,0.000000,300.000000,0.000000,197411.000000,200102.000000
75%,6.066580e+08,1.000000,3.000000,2.990000,3.600000e+05,60.000000,0.000000,41.690000,0.800000,31.910000,791.000000,0.000000,324.000000,0.000000,198402.000000,200807.000000
max,7.177212e+08,1.000000,23.000000,6.100000,4.550000e+06,120.000000,1.000000,999.990000,0.981370,999.990000,860.000000,1.000000,378.000000,1.000000,201808.000000,201808.000000
